In [25]:
import numpy as np
import pandas as pd
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics.pairwise import pairwise_distances

In [26]:
def intra_cluster_distance(X,centers,n_clusters):
    result=np.zeros(n_clusters)
    for i in range(n_clusters):
        result[i]=np.sqrt(np.power(X[X['cluster']==i].drop(['cluster'],axis=1).as_matrix()-centers[i],2).sum(axis=1)).mean()
    return result

In [27]:
traindata=pd.read_csv('train.csv')#reading the data
testdata=pd.read_csv('test.csv')#reading the data

traindata=traindata.drop('ID_Customer',axis=1)
test_ids=testdata['ID_Customer'] #Nos lo guardamos para submision
testdata=testdata.drop('ID_Customer',axis=1)

print('Number of rows and columns of the training set: ',traindata.shape)
print('Number of rows and columns of the test set: ',testdata.shape)

X_traindata=traindata.drop('Poder_Adquisitivo',axis=1)
y_traindata=traindata['Poder_Adquisitivo']


Number of rows and columns of the training set:  (363834, 88)
Number of rows and columns of the test set:  (156315, 87)


In [28]:
n_clusters=5
kmeans=MiniBatchKMeans(n_clusters)

traindata['cluster']=kmeans.fit_predict(X_traindata)



res_table=traindata.groupby(['cluster'])['Poder_Adquisitivo'].describe().sort_values('mean')
res_table['new_cluster']=np.arange(n_clusters)

map_dic=dict(zip(res_table.index,res_table['new_cluster']))



centers=pd.DataFrame(kmeans.cluster_centers_)
centers['new_cluster']=centers.index
centers=centers.set_index(centers['new_cluster'].map(map_dic))
centers=centers.drop(['new_cluster'],axis=1).sort_index().as_matrix()


traindata['cluster']=traindata['cluster'].map(map_dic)
X_traindata['cluster']=traindata['cluster']


In [29]:
res_table

,count,mean,std,min,25%,50%,75%,max,new_cluster
cluster,,,,,,,,,
0,293910.0,13631.555719,16170.271169,3600.960000,8670.922500,11570.185000,15919.647500,5040000.00,0
4,24896.0,22472.841889,25456.408945,3634.080000,13913.524625,19091.585000,26107.535000,1636127.59,1
3,34732.0,26384.779780,33586.334390,5418.104083,16965.787729,21598.006542,29190.903833,2967257.81,2
1,3454.0,35633.262800,65202.529511,3748.500000,18878.622500,26439.570000,36325.455000,2005965.28,3
2,6842.0,53969.568397,86873.836158,12509.395333,29847.498833,38215.169500,51488.754313,2110209.80,4


In [30]:
df=pd.DataFrame(pairwise_distances(centers))
df['intra_cluster_distance']=intra_cluster_distance(X_traindata,centers,n_clusters)
df

,0,1,2,3,4,intra_cluster_distance
0,0.000000,129690.538840,84722.929589,294508.757103,249153.711966,13303.739658
1,129690.538840,0.000000,153670.395267,165640.488395,279067.583832,45587.911655
2,84722.929589,153670.395267,0.000000,306238.316516,169071.761558,54038.099250
3,294508.757103,165640.488395,306238.316516,0.000000,385021.519838,90421.292961
4,249153.711966,279067.583832,169071.761558,385021.519838,0.000000,184093.048909


In [31]:
traindata[['Poder_Adquisitivo','cluster']].corr()

,Poder_Adquisitivo,cluster
Poder_Adquisitivo,1.000000,0.278039
cluster,0.278039,1.000000


In [32]:
traindata.to_csv('train_cluster.csv',sep=',',index=False)
#testdata.to_csv('test.csv',sep=',')